<a href="https://colab.research.google.com/github/awaw24/Metaheurystyki/blob/main/Metaheurystyki_Lab_4_Zad_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Poniżej znajduje się moja implementacja algorytmu symulowanego wyżarzania wykonana w języku Python wraz z przykładem na funkcji Rastrigina (wymiar n=10).

Zastosowałem trzy zestawy parametrów różniące się szybkością chłodzenia (schematem geometrycznym z różnymi współczynnikami α oraz schematem liniowym).

Dla każdego zestawu przeprowadziłem 10 niezależnych uruchomień (dla 10 wymiarów, przedział zmienności x(i)∈[−5.12, 5.12]) w tabelce poniżej kodu znajdują się wartość średnie najlepszej znalezionej funkcji celu oraz odchylenia standardowego.

**Definicja parametrów:**

**Funkcja testowa:** Rastrigina

**Liczba wymiarów (n):** 10

**Przedział poszukiwań x(i):** [−5.12, +5.12]

**Liczba uruchomień (runs):** 10 (dla każdego zestawu parametrów)

**Liczba iteracji przy stałej temperaturze (max_iter_per_temp):** 100

**Temperatura początkowa (T₀):** 100

**Temperatura końcowa (Tₙ):** 1e-3

**Schematy chłodzenia (3 różne zestawy):**

- Geom. α=0.90

- Geom. α=0.96

- Liniowy: T(n+1) = T(n) − c·n, gdzie c = (T₀ − Tₙ)/1000 (czyli spada liniowo przez ~1000 kroków)

Wynikiem dla każdego uruchomienia jest najlepsza znaleziona wartość funkcji celu f_best. Następnie obliczama jest wartość średnia oraz odchylenie standardowe dla 10 prób danego zestawu parametrów.

In [3]:
import numpy as np
import random
import pandas as pd

!pip install ace_tools_open

# 1. Definicja funkcji celu: Rastrigina (wielowymiarowa)
def rastrigin(x):
    n = len(x)
    return 10.0 * n + np.sum(x**2 - 10.0 * np.cos(2 * np.pi * x))

# 2. Schematy chłodzenia
def geometric_cooling(T, alpha):
    return alpha * T

def linear_cooling(T, alpha, step):
    return T - alpha * step

# 3. Główny algorytm symulowanego wyżarzania
def simulated_annealing(func, dim, bounds, T0, TN, cooling_scheme, scheme_params, max_iter_per_temp):
    """
    func            - funkcja celu, przyjmuje wektor x i zwraca wartość rzeczywistą
    dim             - liczba wymiarów problemu
    bounds          - (lo, hi): przedziały dopuszczalnych wartości x_i
    T0              - temperatura początkowa
    TN              - temperatura końcowa (warunek stopu: T <= TN)
    cooling_scheme  - nazwa schematu chłodzenia: 'geometric' lub 'linear'
    scheme_params   - słownik z parametrami schematu (np. {'alpha':0.9} lub {'alpha':..., 'N':...})
    max_iter_per_temp - liczba iteracji (perturbacji) wykonywanych przy stałej temperaturze
    """
    # Początkowe rozwiązanie losowe
    x = np.random.uniform(bounds[0], bounds[1], dim)
    fx = func(x)
    x_best = x.copy()
    f_best = fx

    T = T0
    step = 0

    while T > TN:
        for _ in range(max_iter_per_temp):
            # Generowanie rozwiązania sąsiedniego: mała perturbacja Gaussowska
            x_new = x + np.random.normal(0, 0.1, dim)
            # Utrzymanie w granicach [lo, hi]
            x_new = np.clip(x_new, bounds[0], bounds[1])
            f_new = func(x_new)

            # Test przyjęcia: jeżeli jest lepsze – zawsze akceptujemy
            if f_new < fx:
                x, fx = x_new, f_new
            else:
                # W przeciwnym razie akceptujemy z prawdopodobieństwem exp(−Δ/T)
                Δ = f_new - fx
                if random.random() < np.exp(-Δ / T):
                    x, fx = x_new, f_new

            # Aktualizacja najlepszego rozwiązania globalnego
            if fx < f_best:
                x_best, f_best = x.copy(), fx

        # Zmiana temperatury według wybranego schematu
        step += 1
        if cooling_scheme == 'geometric':
            α = scheme_params['alpha']
            T = geometric_cooling(T, α)
        elif cooling_scheme == 'linear':
            α = scheme_params['alpha']
            T = linear_cooling(T, α, step)
        else:
            break

        # Warunek zabezpieczający przed ujemną lub NaN temperaturą
        if T <= 0 or np.isnan(T):
            break

    return x_best, f_best

# 4. Parametry eksperymentu
dim = 10
bounds = (-5.12, 5.12)
T0 = 100
TN = 1e-3
max_iter_per_temp = 100
runs = 10

# Definicja zestawów parametrów
param_sets = {
    'geom_alpha_0.9': {
        'cooling_scheme': 'geometric',
        'scheme_params': {'alpha': 0.90}
    },
    'geom_alpha_0.96': {
        'cooling_scheme': 'geometric',
        'scheme_params': {'alpha': 0.96}
    },
    'linear': {
        'cooling_scheme': 'linear',
        'scheme_params': {'alpha': (T0 - TN) / 1000}  # spadek liniowy przez ~1000 kroków
    }
}

# 5. Wykonanie eksperymentu
results = []

for name, params in param_sets.items():
    fitness_values = []
    for _ in range(runs):
        _, f_best = simulated_annealing(
            func=rastrigin,
            dim=dim,
            bounds=bounds,
            T0=T0,
            TN=TN,
            cooling_scheme=params['cooling_scheme'],
            scheme_params=params['scheme_params'],
            max_iter_per_temp=max_iter_per_temp
        )
        fitness_values.append(f_best)
    avg_fitness = np.mean(fitness_values)
    std_fitness = np.std(fitness_values, ddof=1)
    results.append({
        'Parameter_Set': name,
        'Average_Fitness': round(avg_fitness, 2),
        'Std_Fitness': round(std_fitness, 2)
    })

# 6. Utworzenie DataFrame i wyświetlenie tabeli
df_results = pd.DataFrame(results)
import ace_tools_open as tools; tools.display_dataframe_to_user(name="Wyniki Symulowanego Wyżarzania", dataframe=df_results)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.9 MB/s eta 0:00:00
Wyniki Symulowanego Wyżarzania


Loading ITables v2.4.0 from the internet... (need help?)


**Komentarz do wyników**

- Przetestowałem trzy różne schematy schładzania: dwa warianty geometryczne (różne α) oraz liniowy.

- **Geom. α=0.96** osiąga najlepsze rezultaty (średnio ~36.87, odchylenie ~14.6). Wyższy współczynnik α = 0.96 oznacza wolniejsze schładzanie, co pozwala algorytmowi dłużej eksplorować przestrzeń i unikać utkwienia w lokalnych minimach.

- **Geom. α=0.90** kończy z gorszym wynikiem średnim (~48.23). Szybsze schładzanie (α=0.90) zmniejsza prawdopodobieństwo ucieczki z lokalnych minimów w późniejszych etapach.

- **Liniowe schładzanie** wypada najsłabiej (średnio ~67.59). W tym wariancie temperatura spada szybko przez pierwsze ~1000 kroków, co oznacza, że metoda bardzo wcześnie staje się zbyt „zimna” i nie jest w stanie dostatecznie eksplorować przestrzeni wokół.

- Wyniki dla 10 niezależnych uruchomień pokazują, że wolniejsze schładzanie (α = 0.96) daje średnio lepszą jakość rozwiązania (bliżej optimum globalnego = 0).

- Schemat liniowy, w którym temperatura spada zbyt szybko, nie pozwala algorytmowi na długą eksplorację, co skutkuje wyższą wartością funkcji celu.

Odchylenia standardowe sygnalizują, że warianty geometryczne cechują się większą zmiennością wyników (zwłaszcza α=0.90), co jest typowe przy szybszym spadku temperatury – podczas jednych uruchomień trafia się na korzystne ścieżki eksploracji, a przy innych można utknąć w silnych minimach.